In [5]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib
import datetime
import os
import sys
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import scipy.stats
from scipy.special import gamma
import json
import urllib.request
from pandas.io.json import json_normalize #package for flattening json in pandas df
sys.path.append('covid_chicago/')
from plotters.aggregate_by_param import *
from plotters.trace_selection import *
from plotters.data_comparison import *

# This notebook reads in the aggregated prevalence files and data files and plots it, to visualize if the prevalence is believable.  It might also combine the prevalence into weeks...

In [23]:
agdf = pd.read_csv('../21-04-reproduce-study/simulation_output/20201216_baseline/trajectoriesDat_aggr.csv',parse_dates=['date'])

In [ ]:
first_day = pd.Timestamp('2020-02-13')
last_day = pd.Timestamp.today()
for region in range(1,12):
    ref_df = load_ref_df(ems_nr=ems)
    plt.figure(figsize=(17,4.5))
    mask = agdf.covid_region == region
    plt.plot(agdf[mask].date, agdf[mask].new_deaths_det_median, color='b',label='median model output')
    plt.fill_between(agdf[mask].date, agdf[mask].new_deaths_det_95CI_lower,
                     agdf[mask].new_deaths_det_95CI_upper,
                     color='b', linewidth=0, alpha=0.2,label='95% CI model output')
    plt.fill_between(agdf[mask].date, agdf[mask].new_deaths_det_50CI_lower, 
                     agdf[mask].new_deaths_det_50CI_upper,
                     color='b', linewidth=0, alpha=0.4,label='50% CI model output')
   
    plt.gca().set_title(f'Region {str(region)}', y=0.8, fontsize=16)

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b\n%y'))
    plt.xlim(first_day, last_day)
    plt.grid(b=True, which='major', color='#999999', linestyle='-', alpha=0.3)
    plt.gca().set_ylabel('New Deaths (model), Deaths',size='large')
    plt.plot(casesdf[casesdf.covid_region==int(region)].date, 
             casesdf[casesdf.covid_region==int(region)].deaths, 'o', color='#303030', 
             linewidth=0, ms=1,label='deaths')
    plt.plot(casesdf[casesdf.covid_region==int(region)].date, 
             casesdf[casesdf.covid_region==int(region)].deaths.rolling(window = 7, center=True).mean(), 
             c='k', alpha=1.0,label='deaths 7-day ave')    
#     ledge=plt.legend(loc='upper right', shadow=True, fancybox=True, numpoints=1,ncol=1,fontsize=18)
    plt.show()


In [8]:
compare_ems('20201216_baseline',ems=1,first_day=pd.Timestamp('2020-02-13'),last_day=pd.Timestamp.today())

TypeError: 'NoneType' object is not iterable

In [28]:
outcome_channels, channels, data_channel_names, titles = get_datacomparison_channels()
first_day = pd.Timestamp('2020-02-13')
last_day = pd.Timestamp.today()
logscale = False
for ems in range(1,12):
    ref_df = load_ref_df(ems_nr=ems)
    fig = plt.figure(figsize=(13, 6))
    palette = sns.color_palette('husl', 8)
    k = 0
    for c, channel in enumerate(['new_deaths_det']):
        ax = fig.add_subplot(2, 3, c + 1)

        mdf = agdf[agdf.covid_region==f'EMS-{str(ems)}']
        ax.plot(mdf['date'], mdf[f'{channel}_median'], color=palette[k])
        ax.fill_between(mdf['date'], mdf[f'{channel}_95CI_lower'], mdf[f'{channel}_95CI_upper'],
                        color=palette[k], linewidth=0, alpha=0.2)
        ax.fill_between(mdf['date'], mdf[f'{channel}_50CI_lower'], mdf[f'{channel}_50CI_upper'],
                        color=palette[k], linewidth=0, alpha=0.4)

        ax.set_title(titles[c], y=0.8, fontsize=12)
        # ax.legend()

        ax.set_title(titles[c], y=0.8, fontsize=12)
        ax.set_xlim(first_day, last_day)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
        ax.grid(b=True, which='major', color='#999999', linestyle='-', alpha=0.3)
        if logscale :
            ax.set_ylim(0.1, ymax)
            ax.set_yscale('log')

        ax.plot(ref_df['date'], ref_df[data_channel_names[c]], 'o', color='#303030', linewidth=0, ms=1)
        ax.plot(ref_df['date'], ref_df[data_channel_names[c]].rolling(window = 7, center=True).mean(), c='k', alpha=1.0)
    fig.tight_layout()
#     if plot_path:
#         plot_name = 'compare_to_data_covidregion_' + str(ems_nr)
#         if logscale == False :
#             plot_name = plot_name + "_nolog"
#         plt.savefig(os.path.join(plot_path, plot_name + '.png'))
#         plt.savefig(os.path.join(plot_path,'pdf',  plot_name + '.pdf'), format='PDF')
